# Intro
This notebook serves as a demonstration of how FMU's can be simulated in Python with [FMPy](https://github.com/CATIA-Systems/FMPy) and how [ModestPy](https://github.com/sdu-cfei/modest-py) can be used for estimating parameters. The model is adapted from the OU44 Modelica library available [here](https://github.com/sdu-cfei/OU44).

The data was collected as a part of the Elforsk project *Data-driven building management and commissioning focusing on energy and indoor climate*. All use of the data and/or this notebook requires proper reference to the author(s):

__Fjerbæk, E. V., & Hviid, C. A. (In preparation) *Benchmarking heating system efficiency in office buildings through grey-box*__

# Import needed libraries

In [ ]:
from modestpy import Estimation
import data_importer as data
import fmpy
from fmpy.util import plot_result
import pandas as pd
import os
from plotly.subplots import make_subplots
import csv

# Import data

In [ ]:
data_path = "Resources/Data/"

# Import external temperature time series from weather data (weather.csv)
weather = pd.read_csv(os.path.join(data_path,"weather.csv"),index_col=0)
weather.index = pd.to_datetime(weather.index)

# Import occupant time series (occupants.csv)
occupants = pd.read_csv(os.path.join(data_path,"occupants.csv"),index_col=0)
occupants.index = pd.to_datetime(occupants.index)

# Import indoor temperature time series (temps_CE.csv)
temps = pd.read_csv(os.path.join(data_path,"temps_CE.csv"),index_col=0)
temps.index = pd.to_datetime(temps.index)

# Group, clean and plot data
Here we group the data into a variables used by ModestPy. We will not go through these - just execute the cells.

## Inputs
Here we restructure measurements inputs

In [ ]:
%%capture
inp = pd.DataFrame(index = weather.index)
inp["time"] = weather.index

inp["solrad"] = weather["solar_rad"]

inp["Tout"] = weather["temp"]

inp["Tvestp"] = data.create_ventilation_stp(weather)["Tvest"]+1

inp["verate"] = data.create_ventilation_rate(inp.index,4,19)

inp["occ"] = occupants["inhouse"]*0.2107
inp["occ"] = inp["occ"]

inp["Tstp"] = 23

inp = inp.reset_index(drop=True)
dates_inp = inp["time"] # X axis with dates - used for plotting
inp["time"] = inp["time"].diff().astype('timedelta64[s]').astype('float').cumsum().fillna(0)
inp = inp.set_index("time")

inp = inp.fillna(method="bfill")

## Ideals
Here we restructure measurements for comparison

In [ ]:
ideal = pd.DataFrame(index = temps["avg"].index)
ideal["time"] = temps["avg"].index

ideal["T"] = temps["avg"]


ideal = ideal.reset_index(drop=True)

ideal["time"] = ideal["time"].diff().astype('timedelta64[s]').astype('float').cumsum().fillna(0)
ideal = ideal.set_index("time")

ideal=ideal.fillna(method="bfill")

## Specify 'est' and 'known'
Here we define the known parameters and bounds for the estimated parameters.

In [ ]:
known = {
         'Vi': 9839,
         'Vinf': 100,
         'RExt': 1.5,
         'maxVent': 33992*0.226,
         'tmass': 1
        }

est =   {
         'shgc': [30, 20, 50],
         'imass':[14,10,18],
         'maxHeat':[1000,0,20000],
        }

ic_param = {
    "TairInit":"T"
}

## Plot inputs
Plot the inputs in a graph to test if data import is successful.

In [ ]:
fig = make_subplots(rows=5, cols=1,
                    shared_xaxes = True,
                    vertical_spacing=0.05,
                   )
row = 0
# Outside temperature
row += 1
fig['layout'][f'yaxis{row}'].update(title="Tout [°C]")
fig.add_scatter(x= dates_inp, y=inp["Tout"],row=row,col=1,name = "Outside - Simulated",legendgroup=str(row), line=dict(color='#636efa', width=1))

# Ventilation setpoint
row += 1
fig['layout'][f'yaxis{row}'].update(title="Ventilation stp [°C]")
fig.add_scatter(x= dates_inp, y=inp["Tvestp"],row=row,col=1,name = "Ventilation stp",legendgroup=str(row), line=dict(color='#636efa', width=1))

# Solar gains
row += 1
fig['layout'][f'yaxis{row}'].update(title="Solar gain [W]")
fig.add_scatter(x= dates_inp, y=inp["solrad"],row=row,col=1,name = "Solar",legendgroup=str(row), line=dict(color='#636efe', width=1))

# Occupancy
row += 1
fig['layout'][f'yaxis{row}'].update(title="Occupancy [-]")
fig.add_scatter(x= dates_inp, y=inp["occ"],row=row,col=1,name = "Occupancy",legendgroup=str(row), line=dict(color='#636efa', width=1))

# verate
row += 1
fig['layout'][f'yaxis{row}'].update(title="Ventilation rate [-]")
fig.add_scatter(x= dates_inp, y=inp["verate"],row=row,col=1,name = "verate",legendgroup=str(row), line=dict(color='#636efa', width=1))

fig.update_layout(
    # width=800,
    height=150*row,
    legend_tracegroupgap = 150*245/300,)


axes_attrs = dict(showgrid=True, gridwidth=1, ticklen=0, gridcolor='LightGrey', linecolor='black', showline=True,
                  zerolinewidth=1, zerolinecolor='LightGrey')
fig.update_xaxes(**axes_attrs)
fig.update_yaxes(**axes_attrs)
fig.update_layout(showlegend=False, margin=dict(t=30, b=0, r=30), plot_bgcolor='rgba(0,0,0,0)')

fig.show()

# Simulate raw model
Here we simulate the model, with the parameters specified in the FMU

In [ ]:
# Rewrite inputs to FMPy format
inp_copy = inp.copy()
inp_np = inp_copy.to_records()
wanted_outputs = ["T","heatFlowSensor.Q_flow","prescribedHeatFlow.Q_flow","re.port_b.Q_flow","occ"]

start_time = 0
stop_time = inp.index[-1]
fmu_path = "Resources/FMUs/ModelicaWorkshop_RoomModel.fmu"

result_realistic=fmpy.simulate_fmu(fmu_path,input=inp_np,output = wanted_outputs,start_time = start_time,stop_time=stop_time,relative_tolerance=1e-5)

plot_result(result_realistic)

# Estimation with ModestPy
We use the following parameters to setup and start an estimation case study:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="float:left">
<thead>
  <tr>
    <th class="tg-0pky">Variable</th>
    <th class="tg-0pky">Value</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">ic_param</td>
    <td class="tg-0pky">ic_param</td>
  </tr>
  <tr>
    <td class="tg-0pky">lp_n</td>
    <td class="tg-0pky">1</td>
  </tr>
  <tr>
    <td class="tg-0pky">lp_frame</td>
    <td class="tg-0pky">(start, end)</td>
  </tr>
  <tr>
    <td class="tg-0pky">vp</td>
    <td class="tg-0pky">(end, end + validation)</td>
  </tr>
  <tr>
    <td class="tg-0pky">ftype</td>
    <td class="tg-0pky">'NRMSE'</td>
  </tr>
  <tr>
    <td class="tg-0pky">default_log</td>
    <td class="tg-0pky">True</td>
  </tr>
  <tr>
    <td class="tg-0pky">logfile</td>
    <td class="tg-0pky">'simple.log'</td>
  </tr>
  <tr>
    <td class="tg-0pky">methods</td>
    <td class="tg-0pky">('MODESTGA',)</td>
  </tr>
  <tr>
    <td class="tg-0pky">generations</td>
    <td class="tg-0pky">15</td>
  </tr>
  <tr>
    <td class="tg-0pky">pop_size</td>
    <td class="tg-0pky">40</td>
  </tr>
  <tr>
    <td class="tg-0pky">trm_size</td>
    <td class="tg-0pky">7</td>
  </tr>
  <tr>
    <td class="tg-0pky">tol</td>
    <td class="tg-0pky">1e-3</td>
  </tr>
  <tr>
    <td class="tg-0pky">workers</td>
    <td class="tg-0pky">3</td>
  </tr>
</tbody>
</table>

In [ ]:
# Parameters
fmu_path = "Resources/FMUs/ModelicaWorkshop_RoomModel.fmu"
workdir = "Results/ModelicaWorkshop_RoomModel"

# Create workdir if it doesn't exist:
if not os.path.exists(workdir):
  os.makedirs(workdir)

start = 0              # Start of learning period
end = 3600*24*14       # End of learning period
validation = 3600*24*14 # Length of validation period

In [ ]:
# Setup estimation
from modestpy import Estimation
session = Estimation(
        workdir,
        fmu_path,
        inp,
        known,
        est,
        ideal,
        ic_param=ic_param,
        lp_n=1,
        lp_frame=(start, end), # Period for learning
        vp=(end, end + validation), # Validation period
        modestga_opts={
            'generations': 15,   # Max. number of generations
            'pop_size': 40,      # Population size
            'trm_size': 7,       # Tournament size
            'tol': 1e-1,         # Absolute tolerance
            'workers': 3         # Number of CPUs to use
        },
        ftype='RMSE', # We use RMSE, because we only use one output for comparison
        default_log=True,
        logfile='simple.log'
    )

In [ ]:
# Run estimation
estimates = session.estimate()

In [ ]:
# Validate solution
err, res = session.validate()

In [ ]:
# Print estimates
print(estimates)

## 2nd try
The solution is bad, and we see that all parameters have met the limit, and therefore we try again. The imass lower limit seems high, so we increase that. With 

In [ ]:
est =   {
         'shgc': [30, 20, 60],
         'imass':[15,1,15],
         'maxHeat':[40000,20000,50000]
        }

In [ ]:
session_2 = Estimation(
        workdir,
        fmu_path,
        inp,
        known,
        est,
        ideal,
        ic_param=ic_param,
        lp_n=1,
        lp_frame=(start, end), # Period for learning
        vp=(end, end + validation), # Validation period
        modestga_opts={
            'generations': 15,   # Max. number of generations
            'pop_size': 40,      # Population size
            'trm_size': 7,       # Tournament size
            'tol': 1e-1,         # Absolute tolerance
            'workers': 3         # Number of CPUs to use
        },
        ftype='RMSE',
        default_log=True,
        logfile='simple.log'
    )

estimates_2 = session_2.estimate()

In [ ]:
err_2, res_2 = session_2.validate()
print(estimates_2)

## 3rd try
We still have values that are limited. We see an offset in the temperature, that can be hard to fix. If we increase the setpoint, we can remove this setpoint. Alternatively, the setpoint could have been a part of the estimates, if it was a parameter in the FMU.

Let's try and replace them.

In [ ]:
inp["Tstp"] = 23.7

est =   {
         'shgc': [30, 20, 60],
         'imass':[15,1,15],
         'maxHeat':[40000,20000,50000]
        }

In [ ]:
session_3 = Estimation(
        workdir,
        fmu_path,
        inp,
        known,
        est,
        ideal,
        ic_param=ic_param,
        lp_n=1,
        lp_frame=(start, end), # Period for learning
        vp=(end, end + validation), # Validation period
        modestga_opts={
            'generations': 15,   # Max. number of generations
            'pop_size': 40,      # Population size
            'trm_size': 7,       # Tournament size
            'tol': 1e-3,         # Absolute tolerance
            'workers': 3         # Number of CPUs to use
        },
        ftype='RMSE',
        default_log=True,
        logfile='simple.log'
    )

estimates_3 = session_3.estimate()

In [ ]:
err_3, res_3 = session_3.validate()
print(estimates_3)

# Simulate, analyze and plot estimated solution
Now we are happy with the results.

To get a detailed view on the simulations, we run the model (as in part 4) with the found with the parameters found in the estimation.

In [ ]:
# Rewrite inputs to FMPy format
inp_copy = inp.copy()
inp_np = inp_copy.to_records()

# Specify the results, we want from the FMU.
wanted_outputs = ["T","Tout","Tret","Tsup","P_temp.y","heatFlowSensor.Q_flow","prescribedHeatFlow1.Q_flow","prescribedHeatFlow.Q_flow","re.port_b.Q_flow","occ","Tstp","filter1.y","verate","filter2.y","solarGainNorth.Q_flow","solarGainSouth.Q_flow","solarGainEast.Q_flow","solarGainWest.Q_flow","rad.Q_flow","airMix_new.port_b.Q_flow","rad.m_flow","max1.y"]

# Combine known and estimates to 'values' variable - if we have already run an estimation in this session, we use that
# - otherwise we import from the 'final.csv' file produced by ModestPy
est_path = "Results/final.csv"
if 'estimates_3' in locals():
    values = {**known, **estimates_3}
else:
    reader = csv.DictReader(open(est_path))
    dictobj = next(reader) 
    dictobj
    values = {**known,
              **dictobj}

# Specify start and stop time
start_time = 0
stop_time = inp.index[-1]

# Run simulation
result_realistic=fmpy.simulate_fmu(fmu_path,input=inp_np,start_values=values,output = wanted_outputs,start_time = start_time,stop_time=stop_time,relative_tolerance=1e-5)

# Clean daata
result_df = pd.DataFrame.from_records(result_realistic, index = "time")
measured_df = pd.concat((ideal,inp),axis=1)

result_df.index = result_df.index/3600
measured_df.index = measured_df.index/3600

result_df["TimeDelta"] = result_df.index.to_series().diff() # Timestep in Hours
measured_df["TimeDelta"] = measured_df.index.to_series().diff() # Timestep in Hours

## Plot simulation results
Blue lines are measurements, orange are simulation results

In [ ]:
dates_result = pd.to_timedelta(result_df.index,'h')+dates_inp[0] # X axis range with dates for simulation results

fig = make_subplots(rows=7, cols=1,
                    shared_xaxes = True,
                    vertical_spacing=0.05,
                    # subplot_titles = ["Room temperature","System temperatures","Occupancy","Solar gains","TRV","Ventilation","Damping","verate"]
                   )


# Outside temperature
row = 1
fig['layout'][f'yaxis{row}'].update(title="Outside temp. [°C]")
fig.add_scatter(x= dates_result, y=result_df["Tout"],row=row,col=1,name = "Outside temperature",legendgroup=str(row), line=dict(color='#636efa', width=1))

# Room temperature
row += 1
fig['layout'][f'yaxis{row}'].update(title="Room temp. [°C]")
fig.add_scatter(x= dates_result, y=result_df["T"],row=row,col=1,name = "Room - Simulated",legendgroup=str(row), line=dict(color='#fcb103', width=1))
fig.add_scatter(x= dates_inp, y=measured_df["T"],row=row,col=1,name = "Room - Measured",legendgroup=str(row), line=dict(color='#636efa', width=1))


# Occupancy
row += 1
fig['layout'][f'yaxis{row}'].update(title="Occupancy [-]")
fig.add_scatter(x= dates_inp, y=measured_df["occ"],row=row,col=1,name = "Occupancy",legendgroup=str(row), line=dict(color='#636efa', width=1))


# ventilation loss
row += 1
fig['layout'][f'yaxis{row}'].update(title="Vent. loss [W]")
fig.add_scatter(x= dates_result, y=result_df["airMix_new.port_b.Q_flow"],row=row,col=1,name = "Ventilation loss",legendgroup=str(row), line=dict(color='#fcb103', width=1))

# transmission loss
row += 1
fig['layout'][f'yaxis{row}'].update(title="Trans. loss [W]")
fig.add_scatter(x= dates_result, y=result_df["re.port_b.Q_flow"],row=row,col=1,name = "Transmission loss",legendgroup=str(row), line=dict(color='#fcb103', width=1))

# solar gain
row += 1
fig['layout'][f'yaxis{row}'].update(title="Solar gain [W]")
fig.add_scatter(x= dates_result, y=result_df["prescribedHeatFlow.Q_flow"],row=row,col=1,name = "Solar gain",legendgroup=str(row), line=dict(color='#fcb103', width=1))

# heating
row += 1
fig['layout'][f'yaxis{row}'].update(title="Heating power [W]")
fig.add_scatter(x= dates_result, y=result_df["prescribedHeatFlow1.Q_flow"],row=row,col=1,name = "Heating power",legendgroup=str(row), line=dict(color='#fcb103', width=1))

fig.update_layout(
    # width=800,
    height=150*row,
    legend_tracegroupgap = 150*240/300,)

axes_attrs = dict(showgrid=True, gridwidth=1, ticklen=0, gridcolor='LightGrey', linecolor='black', showline=True,
                  zerolinewidth=1, zerolinecolor='LightGrey')

fig.update_xaxes(**axes_attrs)
fig.update_yaxes(**axes_attrs)

fig.update_layout(showlegend=False, margin=dict(t=30, b=0, r=30), plot_bgcolor='rgba(0,0,0,0)')

fig.show()